In [ ]:
import serial
import time

import sys
from PyQt6.QtWidgets import *
from PyQt6.QtGui import *
from PyQt6 import uic

In [ ]:
ser = serial.Serial('/dev/ttyACM0', 9600)

while True:
    c = input()

    if c == 'q':
        break
    if c == '1':
        ser.write(b'1')
    if c == '2':
        ser.write(b'2')
    elif c == '3':
        ser.write(b'3')
    elif c == '4':
        ser.write(b'4')
    if c == '5':
        ser.write(b'5')

ser.close()

In [ ]:
import serial
import time

ser = serial.Serial('/dev/ttyACM0', 9600)  # 포트는 PC에 맞게 수정
time.sleep(2)  # 아두이노 리셋 기다림

while True:
    key = input("Enter key (w/s/a/d): ").strip()
    if key in ['w', 's', 'a', 'd', 'x', 'q']:
        if key == 'q':
            ser.write(b'x')
            break

        ser.write(key.encode())

ser.close()

In [ ]:
import sys
from PyQt6.QtWidgets import *
from PyQt6.QtGui import *
from PyQt6 import uic

# ui 파일 연결 - 코드 파일과 같은 폴더 내에 위치
from_class = uic.loadUiType("calculator.ui")[0]

# 화면 클래스
class WindowClass(QMainWindow, from_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.setWindowTitle("CALCULATOR")

        # buttons
        self.FOWARD.clicked.connect(self.FWD)
        self.BACK.clicked.connect(self.BACK)
        self.RIGHT.clicked.connect(self.RIGHT)
        self.LEFT.clicked.connect(self.LEFT)

    def FWD(self):
        print('1')

    def BACK(self):
        print('2')

    def RIGHT(self):
        print('3')

    def LEFT(self):
        print('4')


# Python 메인 함수
if __name__ == "__main__":
    app = QApplication(sys.argv) # 프로그램 실행
    myWindow = WindowClass()     # 화면 클래스 생성
    myWindow.show()              # 프로그램 화면 보이기

    sys.exit(app.exec())         # 프로그램 종료까지 동작

In [1]:
import sys
import serial
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout
from PyQt6.QtCore import Qt

# 시리얼 포트와 속도 설정
ser = serial.Serial('/dev/ttyACM1', 9600, timeout=1)  # 리눅스는 '/dev/ttyUSB0'

class RCController(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("RC카 컨트롤러")
        self.setFixedSize(300, 300)

        # 버튼 생성
        self.forwardBtn = QPushButton('전진')
        self.backwardBtn = QPushButton('후진')
        self.leftBtn = QPushButton('좌회전')
        self.rightBtn = QPushButton('우회전')
        self.stopBtn = QPushButton('정지')

        layout = QVBoxLayout()
        layout.addWidget(self.forwardBtn)
        layout.addWidget(self.backwardBtn)
        layout.addWidget(self.leftBtn)
        layout.addWidget(self.rightBtn)
        layout.addWidget(self.stopBtn)
        self.setLayout(layout)

        # 버튼 이벤트 연결
        self.forwardBtn.clicked.connect(lambda: self.sendCommand('w'))
        self.backwardBtn.clicked.connect(lambda: self.sendCommand('s'))
        self.leftBtn.clicked.connect(lambda: self.sendCommand('a'))
        self.rightBtn.clicked.connect(lambda: self.sendCommand('d'))
        self.stopBtn.clicked.connect(lambda: self.sendCommand('x'))

    def sendCommand(self, cmd):
        ser.write(cmd.encode())

    # 키보드 제어도 추가
    def keyPressEvent(self, event):
        key_map = {
            Qt.Key.Key_W: 'w',
            Qt.Key.Key_S: 's',
            Qt.Key.Key_A: 'a',
            Qt.Key.Key_D: 'd',
            Qt.Key.Key_Space: 'x'
        }
        if event.key() in key_map:
            self.sendCommand(key_map[event.key()])

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RCController()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

/home/lee/venv/autodrive_venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import serial
from PyQt6.QtWidgets import QApplication, QWidget
from PyQt6.QtCore import Qt, QTimer

# 시리얼 포트 설정
ser = serial.Serial('/dev/ttyACM1', 9600, timeout=1)

class RCController(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("RC카 키보드+동시입력 제어")
        self.setFixedSize(200, 200)

        # 상태 추적
        self.keys_pressed = set()
        self.servo_angle = 90
        self.servo_dir = 0  # -1: 좌, +1: 우, 0: 정지

        # 주기적 업데이트 타이머
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_command)
        self.timer.start(50)  # 20fps 정도

    def keyPressEvent(self, event):
        self.keys_pressed.add(event.key())

    def keyReleaseEvent(self, event):
        self.keys_pressed.discard(event.key())

    def update_command(self):
        # === 1. 전진/후진 제어 ===
        if Qt.Key.Key_W in self.keys_pressed:
            ser.write(b'F')
        elif Qt.Key.Key_S in self.keys_pressed:
            ser.write(b'B')
        else:
            ser.write(b'S')

        # === 2. 조향 각도 변경 ===
        if Qt.Key.Key_A in self.keys_pressed and Qt.Key.Key_D not in self.keys_pressed:
            self.servo_dir = -1
        elif Qt.Key.Key_D in self.keys_pressed and Qt.Key.Key_A not in self.keys_pressed:
            self.servo_dir = 1
        else:
            self.servo_dir = 0  # 중립 (누르고 있지 않음)

        if self.servo_dir != 0:
            self.servo_angle += self.servo_dir * 3
            self.servo_angle = max(45, min(135, self.servo_angle))
            command = f"X{self.servo_angle}\n"
            ser.write(command.encode())

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RCController()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

/home/lee/venv/autodrive_venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel
from PyQt6.QtCore import Qt, QTimer
import serial, sys

ser = serial.Serial('COM3', 9600, timeout=1)

class RCController(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("RC카 컨트롤러")
        self.setFixedSize(300, 300)

        self.label = QLabel("연결됨 (COM3)")
        self.forwardBtn = QPushButton("전진")
        self.backwardBtn = QPushButton("후진")
        self.leftBtn = QPushButton("좌회전")
        self.rightBtn = QPushButton("우회전")
        self.stopBtn = QPushButton("정지")

        layout = QVBoxLayout()
        layout.addWidget(self.label)
        layout.addWidget(self.forwardBtn)
        layout.addWidget(self.backwardBtn)
        layout.addWidget(self.leftBtn)
        layout.addWidget(self.rightBtn)
        layout.addWidget(self.stopBtn)
        self.setLayout(layout)

        # 버튼 이벤트
        self.forwardBtn.pressed.connect(lambda: self.sendCommand("F"))
        self.forwardBtn.released.connect(lambda: self.sendCommand("S"))
        self.backwardBtn.pressed.connect(lambda: self.sendCommand("B"))
        self.backwardBtn.released.connect(lambda: self.sendCommand("S"))
        self.leftBtn.pressed.connect(lambda: self.adjustServo(-1))
        self.leftBtn.released.connect(lambda: self.stopServo())
        self.rightBtn.pressed.connect(lambda: self.adjustServo(1))
        self.rightBtn.released.connect(lambda: self.stopServo())
        self.stopBtn.clicked.connect(lambda: self.sendCommand("S"))

        self.keys_pressed = set()
        self.servo_angle = 90
        self.servo_timer = QTimer()
        self.servo_timer.timeout.connect(self.update_servo)

    def sendCommand(self, cmd):
        ser.write(cmd.encode())

    def adjustServo(self, direction):
        self.servo_direction = direction
        self.servo_timer.start(100)

    def stopServo(self):
        self.servo_timer.stop()

    def update_servo(self):
        self.servo_angle += self.servo_direction * 3
        self.servo_angle = max(45, min(135, self.servo_angle))
        ser.write(f"X{self.servo_angle}\n".encode())

    def keyPressEvent(self, event):
        self.keys_pressed.add(event.key())

    def keyReleaseEvent(self, event):
        self.keys_pressed.discard(event.key())

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RCController()
    window.show()
    sys.exit(app.exec())


In [ ]:
import sys
import serial
from PyQt6.QtWidgets import QApplication, QWidget
from PyQt6.QtCore import Qt, QTimer

# ser = serial.Serial('/dev/ttyACM1', 9600, timeout=1)

class RCController(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("RC카 제어기")
        self.setFixedSize(200, 200)

        self.ser = serial.Serial('/dev/ttyACM0', 9600, timeout=1)

        self.keys_pressed = set()
        self.servo_angle = 90
        self.servo_dir = 0

        self.timer = QTimer()
        self.timer.timeout.connect(self.update_command)
        self.timer.start(50)  # 20 FPS

    def keyPressEvent(self, event):
        self.keys_pressed.add(event.key())

    def keyReleaseEvent(self, event):
        self.keys_pressed.discard(event.key())

    def update_command(self):
        # 모터 제어 (전진/후진)
        if Qt.Key.Key_W in self.keys_pressed:
            self.ser.write(b'F\n')
        elif Qt.Key.Key_S in self.keys_pressed:
            self.ser.write(b'B\n')
        else:
            self.ser.write(b'S\n')

        # 서보 조향 제어
        if Qt.Key.Key_A in self.keys_pressed and Qt.Key.Key_D not in self.keys_pressed:
            self.servo_dir = -1
        elif Qt.Key.Key_D in self.keys_pressed and Qt.Key.Key_A not in self.keys_pressed:
            self.servo_dir = 1
        else:
            self.servo_dir = 0

        if self.servo_dir != 0:
            self.servo_angle += self.servo_dir * 3
            self.servo_angle = max(45, min(135, self.servo_angle))
            command = f"X{self.servo_angle}\n"
            self.ser.write(command.encode())

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RCController()
    window.show()
    sys.exit(app.exec())


SystemExit: 0

/home/lee/venv/autodrive_venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sys
import serial
from PyQt6.QtWidgets import (
    QApplication, QWidget, QPushButton, QVBoxLayout, QHBoxLayout, QLabel
)
from PyQt6.QtCore import Qt, QTimer

# 리눅스 시리얼 포트
ser = serial.Serial('/dev/ttyACM1', 9600, timeout=1)

class RCController(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("RC카 GUI 컨트롤러")
        self.setFixedSize(300, 250)

        self.keys_pressed = set()
        self.servo_angle = 90
        self.servo_dir = 0

        # GUI 구성
        self.status_label = QLabel("상태: 정지")
        self.status_label.setAlignment(Qt.AlignmentFlag.AlignCenter)

        self.forward_button = QPushButton("↑ 전진")
        self.backward_button = QPushButton("↓ 후진")
        self.left_button = QPushButton("← 좌회전")
        self.right_button = QPushButton("→ 우회전")

        # 버튼 이벤트 연결
        self.forward_button.pressed.connect(lambda: self.keys_pressed.add(Qt.Key.Key_W))
        self.forward_button.released.connect(lambda: self.keys_pressed.discard(Qt.Key.Key_W))

        self.backward_button.pressed.connect(lambda: self.keys_pressed.add(Qt.Key.Key_S))
        self.backward_button.released.connect(lambda: self.keys_pressed.discard(Qt.Key.Key_S))

        self.left_button.pressed.connect(lambda: self.keys_pressed.add(Qt.Key.Key_A))
        self.left_button.released.connect(lambda: self.keys_pressed.discard(Qt.Key.Key_A))

        self.right_button.pressed.connect(lambda: self.keys_pressed.add(Qt.Key.Key_D))
        self.right_button.released.connect(lambda: self.keys_pressed.discard(Qt.Key.Key_D))

        # 레이아웃 구성
        vbox = QVBoxLayout()
        vbox.addWidget(self.status_label)

        hbox1 = QHBoxLayout()
        hbox1.addStretch()
        hbox1.addWidget(self.forward_button)
        hbox1.addStretch()

        hbox2 = QHBoxLayout()
        hbox2.addWidget(self.left_button)
        hbox2.addWidget(self.backward_button)
        hbox2.addWidget(self.right_button)

        vbox.addLayout(hbox1)
        vbox.addLayout(hbox2)

        self.setLayout(vbox)

        # 주기적 업데이트 타이머
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_command)
        self.timer.start(50)

    def keyPressEvent(self, event):
        self.keys_pressed.add(event.key())

    def keyReleaseEvent(self, event):
        self.keys_pressed.discard(event.key())

    def update_command(self):
        # === 1. 전진/후진 제어 ===
        if Qt.Key.Key_W in self.keys_pressed:
            ser.write(b'F\n')
            self.status_label.setText("상태: 전진")
        elif Qt.Key.Key_S in self.keys_pressed:
            ser.write(b'B\n')
            self.status_label.setText("상태: 후진")
        else:
            ser.write(b'S\n')
            self.status_label.setText("상태: 정지")

        # === 2. 조향 제어 ===
        if Qt.Key.Key_A in self.keys_pressed and Qt.Key.Key_D not in self.keys_pressed:
            self.servo_dir = -1
        elif Qt.Key.Key_D in self.keys_pressed and Qt.Key.Key_A not in self.keys_pressed:
            self.servo_dir = 1
        else:
            self.servo_dir = 0

        if self.servo_dir != 0:
            self.servo_angle += self.servo_dir * 3
            self.servo_angle = max(45, min(135, self.servo_angle))
            command = f"X{self.servo_angle}\n"
            ser.write(command.encode())

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = RCController()
    window.show()
    sys.exit(app.exec())
